Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "08")
cd(ProjDir)

### snippet 10.1

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_01 = "
data{
    int N;
    int pulled_left[N];
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    pulled_left ~ binomial( 1 , inv_logit(a) );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_01",
monitors = ["a"],
model=m_10_01, output_format=:mcmcchains);

Input data for cmdstan

In [6]:
m_10_01_data = Dict("N" => size(df, 1),
"pulled_left" => df[:pulled_left]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_01_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
  mean   sd  5.5% 94.5% n_eff Rhat
a 0.32 0.09 0.18  0.46   166    1
";

Describe the draws

In [8]:
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std       │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64   │ Float64    │ Float64    │ Any    │
├─────┼────────────┼──────────┼───────────┼────────────┼────────────┼────────┤
│ 1   │ a          │ 0.322822 │ 0.0893878 │ 0.00141335 │ 0.00239076 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%     │ 25.0%    │ 50.0%   │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64  │ Float64  │ Float64 │ Float64  │ Float64  │
├─────┼────────────┼──────────┼──────────┼─────────┼──────────┼──────────┤
│ 1   │ a          │ 0.153895 │ 0.260501 │ 0.32397 │ 0.383182 │ 0.495047 │


End of `10/m10.01s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*